In [1]:
library(rethinking)

Loading required package: rstan
Warning message:
"package 'rstan' was built under R version 3.3.2"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.3.2"Loading required package: StanHeaders
Warning message:
"package 'StanHeaders' was built under R version 3.3.2"rstan (Version 2.13.2, packaged: 2016-12-18 07:04:42 UTC, GitRev: 5fa1e80eb817)
For execution on a local, multicore CPU with excess RAM we recommend calling
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())
Loading required package: parallel
rethinking (Version 1.59)


In [56]:
data(Howell1) 
d <- Howell1
mean(d$weight) 
sd(d$weight)
weig.mmu <- 35.6106175924632
weig.std <- 14.71917817928
mean(d$height)
sd(d$height)
d$weight <- (d$weight - mean(d$weight)) /sd(d$weight)
d$height <- (d$height - mean(d$height)) /sd(d$height)
d$age <- (d$age - mean(d$age)) /sd(d$age)
d$male <- (d$male - mean(d$male)) /sd(d$male)
mean(d$weight) 
sd(d$weight)

[1] 35.61062

[1] 14.71918

[1] 138.2636

[1] 27.60245

[1] -2.712698e-18

[1] 1

In [ ]:
# 4.7.12. Missing heights. 
# The weights listed below were recorded
# in the !Kung census, but heights were not recorded for these individuals.
# Provide predicted heights and 90% confidence intervals (HPDI or PI)for each of these individuals.
# That is, fill in the table below,using model-based predictions.
# c(46.95, 43.72, 64.78, 32.59, 54.63)

In [45]:
d2 <- d[ d$age >= -0.546799754299813  , ]
# fit model 
m4.3 <- map( 
    alist( 
        height ~ dnorm( mu , sigma ),
        mu <- a + b*weight ,#+ b1*male + b2*age ,
        a ~ dnorm( 156 , 100 ) ,
        b ~ dnorm( 0 , 10 ) , 
        #b1 ~ dnorm( 0 , 10 ) ,
        #b2 ~ dnorm( 0 , 10 ) ,        
        sigma ~ dunif( 0 , 50 ) ) ,
    data=d2 , start=list( a=mean(d2$height) , sigma=sd(d2$height) , b=0 ) )

In [68]:
# define sequence of weights to compute predictions for 
# these values will be on the horizontal axis 
weight.seq1 <- c(46.95, 43.72, 64.78, 32.59, 54.63)
weight.seq <- (weight.seq-weig.mmu)/weig.std

# use link to compute mu 
# for each sample from posterior 
# and for each weight in weight.seq 
mu <- link( m4.3 , data=list(weight=weight.seq) )
str(mu)
sim.height <- sim( m4.3 , data=list(weight=weight.seq) ) 
str(sim.height)

[ 1000 / 1000 ]
 num [1:1000, 1:5] -0.886 -0.872 -0.819 -0.925 -0.983 ...
[ 1000 / 1000 ]
 num [1:1000, 1:5] -0.993 -0.886 -0.562 -0.96 -0.834 ...


In [69]:
heightMU <- 138.263596323529
heightSTD <- 27.6024476405647

In [70]:
c(apply( mu , 2 , mean ) )*heightSTD + heightMU 
c(apply( mu , 2 , HPDI ) )*heightSTD + heightMU 

#Missing Height Answer:
c(apply( sim.height , 2 , mean ) )*heightSTD + heightMU 
c(apply( sim.height , 2 , HPDI ) )*heightSTD + heightMU 

[1] 114.6283 114.4300 115.7228 113.7467 115.0997

[1] 112.0648 117.7178 111.8546 117.5349 113.2248 118.7284 111.1251 116.9045
 [9] 112.5645 118.1528

[1] 114.7390 114.3522 115.6557 113.7078 115.0789

[1] 105.6969 122.8734 106.5063 123.8775 107.3269 124.0724 105.8760 122.8575
 [9] 106.3205 123.1584